In [2]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [3]:
from pyspark.sql import Row

In [4]:
file_location1 = "/FileStore/tables/aries_crash_data_2016.csv"
file_location2 = "/FileStore/tables/aries_crash_data_2017.csv"
file_location3 = "/FileStore/tables/aries_crash_data_2018.csv"

file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","
quotechar = '"'

# The applied options are for CSV files. For other file types, these will be ignored.
df_2016 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("quotechar", quotechar)\
  .load(file_location1)
  
df_2017 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("quotechar", quotechar)\
  .load(file_location2)

df_2018 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("quotechar", quotechar)\
  .load(file_location3)


appended1 = df_2016.union(df_2017)
complete_set = appended1.union(df_2018)
#complete_set.count()

df = complete_set

In [5]:
#df = spark.read.csv('FileStore/tables/Complete_Aries.csv', header=True, inferSchema=True)

In [7]:
df_gender = df.select(df['GENDERCDE'])

In [8]:
df_gender.take(3)

Out[9]: [Row(GENDERCDE=None), Row(GENDERCDE=None), Row(GENDERCDE=None)]

In [9]:
df_gender_filtered = df_gender.filter(df_gender['GENDERCDE'].isNotNull())

In [10]:
from pyspark.sql import functions as sf
from pyspark.sql.functions import col, avg
df_gender_count = df_gender_filtered.groupby('GENDERCDE').agg(sf.count('GENDERCDE'))
display(df_gender_count.sort(col('count(GENDERCDE)').desc()).collect())

GENDERCDE,count(GENDERCDE)
M,592915
F,480922


In [11]:
## Not yet account for the duplicate of entries when two or more people in the same vehicle

df_age_grp = df.select(df['AGE_GRP'])
df_age_grp_flt = df_age_grp.filter(df_age_grp['AGE_GRP'].isNotNull())
df_age_grp_count = df_age_grp_flt.groupby('AGE_GRP').agg(sf.count('AGE_GRP'))
display(df_age_grp_count.sort(col('count(AGE_GRP)').desc()).collect())

AGE_GRP,count(AGE_GRP)
15-24 Years,253683
25-34 Years,221611
35-44 Years,174167
45-54 Years,159306
55-64 Years,137096
65-74 Years,76922
75-84 Years,32522
>=85 Years,9296
5-14 Years,8153
1-4 Years,2172


In [12]:
## create sql table
df.createOrReplaceTempView("datatable")

In [13]:
df1 = df.dropDuplicates(['INDEXING_NUMBER'])
df1.count()

Out[3]: 1393621

In [14]:
## get incident type on duplicate-free dataframe
## Not yet account for the duplicate of entries when two or more people in the same vehicle

df_manner = df1.select(df1['MANNERCOLLDESCR'])
df_manner_flt = df_manner.filter(df_manner['MANNERCOLLDESCR'].isNotNull())
df_manner_count = df_manner_flt.groupby('MANNERCOLLDESCR').agg(sf.count('MANNERCOLLDESCR'))
display(df_manner_count.sort(col('count(MANNERCOLLDESCR)').desc()).collect())

MANNERCOLLDESCR,count(MANNERCOLLDESCR)
Rear End,352858
Ran off Road,195322
Right Angle,184784
Same Direction Sideswipe,143481
Backing,126134
Other (Explain in Narrative),87306
Left Turn,71722
Head On,35857
Opposite Direction Sideswipe,31255
Right Turn,19142


In [15]:
df_weather = df1.select(df1['WEATHERDESCR'])
df_weather_flt = df_weather.filter(df_weather['WEATHERDESCR'].isNotNull())
df_weather_count = df_weather_flt.groupby('WEATHERDESCR').agg(sf.count('WEATHERDESCR'))
display(df_weather_count.sort(col('count(WEATHERDESCR)').desc()).collect())

WEATHERDESCR,count(WEATHERDESCR)
Clear,893931
Cloudy,268246
Rain,143543
Snow,49391
Blowing Sand/Soil/Snow,14456
Sleet/Hail/Freezing Rain,12169
Fog/Smoke/Smog,6992
Severe Cross Wind,1835


In [16]:
df_pre_coll = df1.select(df1['PRECOLLACTDESCR'])
df_pre_flt = df_pre_coll.filter(df_pre_coll['PRECOLLACTDESCR'].isNotNull())
df_pre_count = df_pre_flt.groupby('PRECOLLACTDESCR').agg(sf.count('PRECOLLACTDESCR'))
display(df_pre_count.sort(col('count(PRECOLLACTDESCR)').desc()).collect())

PRECOLLACTDESCR,count(PRECOLLACTDESCR)
Going Straight,746861
Slowing or Stopped in Traffic,239057
Turning Left,88420
Parked,76404
Backing,75150
Turning Right,45012
Changing Lanes,21629
Leaving Traffic Lane,14668
Entering Traffic Lane,14342
Starting in Traffic,11357


In [17]:
# Find frequency of incident on weekdays

df_day_coll = df1.select(df1['COLLISION_DAY'])
df_day_flt = df_day_coll.filter(df_day_coll['COLLISION_DAY'].isNotNull())
df_day_count = df_day_flt.groupby('COLLISION_DAY').agg(sf.count('COLLISION_DAY'))
display(df_day_count.collect())

COLLISION_DAY,count(COLLISION_DAY)
MONDAY,66959
TUESDAY,67925
FRIDAY,78042
SUNDAY,46337
WEDNESDAY,71425
SATURDAY,58067
THURSDAY,69763


In [18]:
df_dead = df1.filter(df['DEADNMB'] > 0)

In [19]:
df_dead_road_type = df_dead.select(df_dead['ROADTYPEDESCR'],df_dead['DEADNMB'])
df_dead_road_type_count = df_dead_road_type.groupby('ROADTYPEDESCR').agg(sf.count('DEADNMB'))
df_dead_road_type_count.take(5)

Out[27]: [Row(ROADTYPEDESCR='Two Lanes (One Way)', count(DEADNMB)=162),
 Row(ROADTYPEDESCR='Two Lanes (Two Way)', count(DEADNMB)=5119),
 Row(ROADTYPEDESCR='Multi-Lanes (One Way)', count(DEADNMB)=93),
 Row(ROADTYPEDESCR=None, count(DEADNMB)=2425),
 Row(ROADTYPEDESCR='Multi-Lane Undivided 2-way left (TwoWay)', count(DEADNMB)=202)]

In [20]:
# get accidents by month

df_month = df1.select(df1['COLLISION_MONTH'])
#df_month_flt = df_month.filter(df_month['COLLISION_MONTH'].isNotNull())
df_month_count = df_month.groupby('COLLISION_MONTH').agg(sf.count('COLLISION_MONTH'))
display(df_month_count.sort(col('count(COLLISION_MONTH)').desc()).collect())

COLLISION_MONTH,count(COLLISION_MONTH)
JAN,43726
OCT,42815
NOV,42349
MAY,39135
DEC,38899
AUG,37912
SEP,37310
JUN,36755
JUL,36619
MAR,35762


In [21]:
# get accidents by day

df_month = df1.select(df1['COLLISION_DAY'])
#df_month_flt = df_month.filter(df_month['COLLISION_MONTH'].isNotNull())
df_month_count = df_month.groupby('COLLISION_DAY').agg(sf.count('COLLISION_DAY'))
display(df_month_count.sort(col('count(COLLISION_DAY)').desc()).collect())

COLLISION_DAY,count(COLLISION_DAY)
FRIDAY,237848
WEDNESDAY,212604
THURSDAY,209283
TUESDAY,207232
MONDAY,198931
SATURDAY,181795
SUNDAY,145928


In [22]:
# get accidents by Light conditions

df_month = df1.select(df1['LIGHTCONDDESCR'])
#df_month_flt = df_month.filter(df_month['COLLISION_MONTH'].isNotNull())
df_month_count = df_month.groupby('LIGHTCONDDESCR').agg(sf.count('LIGHTCONDDESCR'))
display(df_month_count.sort(col('count(LIGHTCONDDESCR)').desc()).collect())

LIGHTCONDDESCR,count(LIGHTCONDDESCR)
Daylight,933237
Dark (Not Lighted),193165
Dark (Lighted),192486
Dawn/Dusk,66198
Unknown,6208
null,0


In [23]:
# get accidents by Weather conditions

df_month = df1.select(df1['WEATHERDESCR'])
#df_month_flt = df_month.filter(df_month['COLLISION_MONTH'].isNotNull())
df_month_count = df_month.groupby('WEATHERDESCR').agg(sf.count('WEATHERDESCR'))
display(df_month_count.sort(col('count(WEATHERDESCR)').desc()).collect())

WEATHERDESCR,count(WEATHERDESCR)
Clear,893931
Cloudy,268246
Rain,143543
Snow,49391
Blowing Sand/Soil/Snow,14456
Sleet/Hail/Freezing Rain,12169
Fog/Smoke/Smog,6992
Severe Cross Wind,1835
null,0


In [24]:
PRIMARYFACTORDESCR

In [25]:
# primary factors for incidents

df_prim = df1.select(df1['PRIMARYFACTORDESCR'])
df_prim_count = df_prim.groupby('PRIMARYFACTORDESCR').agg(sf.count('PRIMARYFACTORDESCR'))
display(df_prim_count.sort(col('count(PRIMARYFACTORDESCR)').desc()).take(10))

PRIMARYFACTORDESCR,count(PRIMARYFACTORDESCR)
FOLLOWING TOO CLOSELY,250240
FAILURE TO YIELD RIGHT OF WAY,229250
UNSAFE BACKING,122967
RAN OFF ROAD,112534
ANIMAL/OBJECT IN ROADWAY,111837
"OTHER (DRIVER),EXPLAIN IN NARRATIVE",91527
UNSAFE LANE MOVEMENT,67680
SPEED TOO FAST FOR WEATHER CONDITIONS,52328
DISREGARD SIGNAL/REG SIGN,52297
IMPROPER TURNING,48158


In [26]:
SPEEDLIMITTXT

In [27]:
# Get speed limit in FATAL accidents

df_speed = df_dead.select(df_dead['SPEEDLIMITTXT'])
df_speed_count = df_speed.groupby('SPEEDLIMITTXT').agg(sf.count('SPEEDLIMITTXT')).sort('SPEEDLIMITTXT', ascending = 0)
display(df_speed_count.sort(col('SPEEDLIMITTXT').desc()).collect())

SPEEDLIMITTXT,count(SPEEDLIMITTXT)
NA,22
70,297
66,1
65,176
60,282
55,2273
50,300
5,8
49,1
45,813


In [28]:
# get deer involve accidents

df_deer = df1.select(df1['DEERNMB'],df1['COUNTYDESCR'])
df_deer_count = df_deer.groupby('COUNTYDESCR').agg(sf.sum('DEERNMB'))
display(df_deer_count.sort(col('sum(DEERNMB)').desc()).collect())

COUNTYDESCR,sum(DEERNMB)
Allen,987
Steuben,963
Kosciusko,912
Noble,747
Elkhart,727
LaPorte,708
Marshall,693
Porter,692
St Joseph,675
Tippecanoe,651


In [29]:
# weather when deer involved accidents

df_deer = df1.select(df1['DEERNMB'],df1['COUNTYDESCR'])
df_deer_count = df_deer.groupby('COUNTYDESCR').agg(sf.sum('DEERNMB'))
display(df_deer_count.sort(col('sum(DEERNMB)').desc()).collect())

In [30]:
df_deer = df1.filter(df1['DEERNMB'] > 0)

df_deer_location = df_deer.select(df_deer['LATDECIMALNMB'], df_deer['LONGDECIMALNMB'])
df_deer_loc_flt1 = df_deer_location.filter(df_deer_location['LATDECIMALNMB']!= 0)
df_deer_loc_flt2 = df_deer_loc_flt1.filter(df_deer_loc_flt1['LONGDECIMALNMB'] != 0)
#df_deer_count = df_deer.groupby('LIGHTCONDDESCR').agg(sf.count('LIGHTCONDDESCR'))
#display(df_deer_count.sort(col('count(LIGHTCONDDESCR)').desc()).collect())


In [31]:
pdf1=df_deer_loc_flt2.toPandas()

In [32]:
file_name = '/dbfs/FileStore/deer_accident_location.csv'
#('/dbfs/FileStore/tables/gridsol.csv', sep=',', header=True, index=False)
#pdf1.to_csv(r'/dbfs/FileStore/deer_accident_location.txt', header=None, index=None, sep=' ', mode='a')

In [33]:
# get accidents location

df_location = df1.select(df1['UNIQUELOCATIONID'])
#df_month_flt = df_month.filter(df_month['COLLISION_MONTH'].isNotNull())
df_location_count = df_location.groupby('UNIQUELOCATIONID').agg(sf.count('UNIQUELOCATIONID'))
display(df_location_count.sort(col('count(UNIQUELOCATIONID)').desc()).collect())

UNIQUELOCATIONID,count(UNIQUELOCATIONID)
I65,591
MAINST,582
I80,513
EASTMAINST,463
SAGAMOREPKYRD,439
I65S,385
GREENRIVERRD,380
STATEST,368
GRAPERD,366
I465N,358


In [34]:
# get county in IN that has the most accidents

df_county = df1.select(df1['COUNTYDESCR'],df1['UNIQUELOCATIONID'])


In [35]:
# get county in IN that has the most deaths in accidents >2
df_dead = df.filter(df['DEADNMB'] > 0)
df_county = df_dead.select(df_dead['COUNTYDESCR'],df_dead['UNIQUELOCATIONID'])
## Process RDD for accident location by county
county_rdd = df_county.rdd.map(tuple)
################
def reduce_function(x):
    output=[]
    if x[0] not in output:
        output.append((x[0],{}))
    for i in x[1]:
        k=i[0]
        v=i[1]
        if k not in output[0][1].keys():
            output[0][1][k]=0
        
        output[0][1][k]+=1
        
    return output
  
############################
def produce_output(x):
    output = []
    if x[0][0] not in output:
        output.append((x[0][0],[]))
    for k, v in x[0][1].items():
        if x[0][0] not in output:
           if v > 2:
                output[0][1].append((k,v))
          #output[0][1].append((k,v))
        
    return output
#############################
  
map_f1 = lambda x: [(a,1) for a in x if a != 1]
#########

countyR1 = county_rdd.map(lambda x: (x[0],(x[1],1)))
countyR2 = countyR1.reduceByKey(lambda k,v: k+v)

countyR2_reduce = countyR2.map(lambda x: (x[0],map_f1(x[1])))
countyR3 = countyR2_reduce.map(reduce_function)
countyR4 = countyR3.map(produce_output)

###########################

def filter_output(x):
  output = []
  for i in x:
    if i[1] != []:
      output.append(i)
    else:
      output.append('')
    
  return output

##########################

countyR5_dead = countyR4.map(filter_output)
########################################
## convert to python list

result = countyR5_dead.collect()
final_result_dead1= []
for i in result:
  if i != ['']:
    final_result_dead1.append(i)
    

In [37]:
# get county in IN that has the most FATAL accidents, DROP individual duplicates >2
df_dead = df1.filter(df1['DEADNMB'] > 0)
df_county = df_dead.select(df_dead['COUNTYDESCR'],df_dead['UNIQUELOCATIONID'])
## Process RDD for accident location by county
county_rdd = df_county.rdd.map(tuple)
################
def reduce_function(x):
    output=[]
    if x[0] not in output:
        output.append((x[0],{}))
    for i in x[1]:
        k=i[0]
        v=i[1]
        if k not in output[0][1].keys():
            output[0][1][k]=0
        
        output[0][1][k]+=1
        
    return output
  
############################
def produce_output(x):
    output = []
    if x[0][0] not in output:
        output.append((x[0][0],[]))
    for k, v in x[0][1].items():
        if x[0][0] not in output:
           if v > 2:
                output[0][1].append((k,v))
          #output[0][1].append((k,v))
        
    return output
#############################
  
map_f1 = lambda x: [(a,1) for a in x if a != 1]
#########

countyR1 = county_rdd.map(lambda x: (x[0],(x[1],1)))
countyR2 = countyR1.reduceByKey(lambda k,v: k+v)

countyR2_reduce = countyR2.map(lambda x: (x[0],map_f1(x[1])))
countyR3 = countyR2_reduce.map(reduce_function)
countyR4 = countyR3.map(produce_output)

###########################

def filter_output(x):
  output = []
  for i in x:
    if i[1] != []:
      output.append(i)
    else:
      output.append('')
    
  return output

##########################

countyR5_dead = countyR4.map(filter_output)
########################################
## convert to python list

result = countyR5_dead.collect()
final_result_dead= []
for i in result:
  if i != ['']:
    final_result_dead.append(i)
    
#final_result

In [39]:
## get total accidents of location that have more than 50 accidents in 2016-18

In [40]:
## getting total accidents of location that have more than 100 accidents 2016-18

df_county = df1.select(df1['COUNTYDESCR'],df1['UNIQUELOCATIONID'])
## Process RDD for accident location by county
county_rdd = df_county.rdd.map(tuple)
################
def reduce_function(x):
    output=[]
    if x[0] not in output:
        output.append((x[0],{}))
    for i in x[1]:
        k=i[0]
        v=i[1]
        if k not in output[0][1].keys():
            output[0][1][k]=0
        
        output[0][1][k]+=1
        
    return output
  
############################
def produce_output(x):
    output = []
    if x[0][0] not in output:
        output.append((x[0][0],[]))
    for k, v in x[0][1].items():
        if x[0][0] not in output:
           if v >= 50:
                output[0][1].append((k,v))
          #output[0][1].append((k,v))
        
    return output
#############################
  
map_f1 = lambda x: [(a,1) for a in x if a != 1]
#########

countyR1 = county_rdd.map(lambda x: (x[0],(x[1],1)))
countyR2 = countyR1.reduceByKey(lambda k,v: k+v)

countyR2_reduce = countyR2.map(lambda x: (x[0],map_f1(x[1])))
countyR3 = countyR2_reduce.map(reduce_function)
countyR4 = countyR3.map(produce_output)

###########################

def filter_output(x):
  output = []
  for i in x:
    if i[1] != []:
      output.append(i)
    else:
      output.append('')
    
  return output

##########################

countyR5_total = countyR4.map(filter_output)
########################################
## convert to python list

result = countyR5_total.collect()
final_result_total= []
for i in result:
  if i != ['']:
    final_result_total.append(i)
    
#final_result

In [41]:
countyR5_dead
countyR5_total

In [42]:
allRDD = sc.union([countyR5_dead,countyR5_total])

In [43]:
allRDD.count()

Out[33]: 185

In [44]:
#record the rate death/fatality rate that higher than 0.3

def get_death_per_accident(a,b):
  output = {}
  
  for j in a:

    for k in b:
      if k[0][0] == j[0][0]:
        #print(k[0]) ## match name of county
        if k[0][0] not in output.keys():
          output[k[0][0]]={}
        for l in k[0][1]:
          for m in j[0][1]:
            if l[0] == m[0]:
              #print(l[1])
            #if l[0] == m[0]:
              ratio = m[1]/l[1]
              
              if ratio > 0.05:
                if l[0] not in output[k[0][0]].keys():
                  output[k[0][0]][l[0]]= 0
                output[k[0][0]][l[0]] = ratio

  return output

In [46]:
# yield result of fatal ratio per location
out = get_death_per_accident(final_result_dead,final_result_total)

In [47]:
# yield result of death per acc per location
out1 = get_death_per_accident(final_result_dead1,final_result_total)

In [48]:
# Percent of fatality per accident in the locations that have more than 50 accidents from 2016-2018 (number of accidents resulted in deaths/ total number of accidents)
out

Out[69]: {'Porter': {},
 'Decatur': {},
 'Boone': {},
 'St Joseph': {},
 'Tippecanoe': {},
 'Vigo': {},
 'Madison': {},
 'Elkhart': {},
 'Wells': {},
 'Delaware': {},
 'Fulton': {},
 'Lake': {},
 'Monroe': {},
 'Gibson': {},
 'Hendricks': {},
 'Lawrence': {},
 'De Kalb': {},
 'Hamilton': {},
 'Starke': {},
 'Huntington': {},
 'Vanderburgh': {},
 'Dearborn': {},
 'Hancock': {},
 'Clinton': {},
 'Jasper': {},
 'Dubois': {},
 'Warrick': {},
 'Putnam': {},
 'Grant': {},
 'Harrison': {},
 'Marion': {},
 'Clark': {},
 'LaPorte': {},
 'Floyd': {},
 'Wayne': {},
 'Johnson': {},
 'Allen': {},
 'Jay': {},
 'Jackson': {},
 'Marshall': {},
 'Whitley': {},
 'Tipton': {},
 'Bartholomew': {},
 'Wabash': {},
 'Cass': {},
 'Adams': {},
 'Howard': {},
 'Steuben': {},
 'Jefferson': {},
 'Kosciusko': {},
 'Morgan': {},
 'Jennings': {}}

In [50]:
#with open('/dbfs/FileStore/total_acc_per_loc_per_county.txt', 'w') as filehandle:
    #for listitem in final_result_total:
        #filehandle.write('%s\n' % listitem)

In [51]:
# Total number of deaths in each location in each county (where locations have more than 50 accidents from 16-18)
final_result_total

Out[70]: [[('Porter',
 [('CALUMETAVE', 156),
 ('MERIDIANRDUS6', 247),
 ('I9419.0', 106),
 ('CALUMETAVEVALEPARKRD', 189),
 ('SR149US6', 229),
 ('MORTHLANDDR', 112),
 ('CR600NSR130', 158),
 ('CENTRALAVEWILLOWCREEKRD', 122),
 ('CR475WUS30', 119),
 ('CENTRALAVE', 201),
 ('US20', 113),
 ('US6', 641),
 ('I94', 135),
 ('WILLOWCREEKRD', 130),
 ('PORTERAVESR49', 157),
 ('INDIANBOUNDARYRD', 182),
 ('CR1100NSR49', 154),
 ('GATEWAYBLVDSR49', 174),
 ('INDIANBOUNDARYRDSR49', 251),
 ('CR200WUS6', 126),
 ('I9426.0', 106),
 ('SWANSONRDUS6', 107),
 ('CR250WSR130', 119),
 ('I94SR249', 108),
 ('CALUMETAVEGLENDALEBLVD', 111),
 ('US6WILLOWCREEKRD', 288),
 ('CR600NSR49', 144),
 ('CR250WUS30', 133),
 ('AIRPORTRDUS6', 115),
 ('CR600WUS30', 116),
 ('LUTERDWILLOWCREEKRD', 116),
 ('SR49US6', 227)])],
 [('Decatur', [('NLINCOLNST', 119)])],
 [('Boone',
 [('WHITESTOWNPKWY', 208),
 ('NLEBANONST', 110),
 ('I65N', 122),
 ('LEBANONST', 100)])],
 [('St Joseph',
 [('EDISONLAKESPKWYGRAPERD', 166),
 ('WESTERNAVE', 117),
 ('CLEVELANDRDRIVERSIDEDR', 102),
 ('WALNUTSTWESTERNAVE', 135),
 ('CLEVELANDRDIRONWOODRD', 141),
 ('US20W', 125),
 ('EDISONLAKESPKWYMAINST', 131),
 ('BITTERSWEETRDCLEVELANDRD', 142),
 ('EUNIVERSITYDR', 108),
 ('IRELANDRDIRONWOODDR', 138),
 ('GRAPERD', 1084),
 ('EDISONRDHICKORYRD', 114),
 ('CLEVELANDRDPORTAGEAVE', 105),
 ('BREMENHWY', 113),
 ('IRONWOODDRSOUTHBENDAVE', 144),
 ('EDISONRDGRAPERD', 139),
 ('IRELANDRDMIAMIST', 105),
 ('US20', 177),
 ('IRELANDRDMICHIGANST', 190),
 ('GRAPERDINDIANRIDGEBLVD', 172),
 ('NMAINST', 122),
 ('BRICKRDGUMWOODRD', 110),
 ('MAINSTSAMPLEST', 103),
 ('MICHIGANSTSAMPLEST', 134),
 ('DOUGLASRDMAINST', 127),
 ('DOUGLASRDGRAPERD', 237),
 ('MICHIGANSTMONROEST', 104),
 ('EIRELANDRD', 118),
 ('HICKORYRD', 148),
 ('I80', 142),
 ('IRELANDRD', 195),
 ('GRAPERDSR23', 121),
 ('PORTAGEAVE', 152),
 ('GRAPERDUNIVERSITYDR', 124),
 ('SR331SR933', 106)])],
 [('Tippecanoe',
 [('MCCARTYLN', 102),
 ('SAGAMOREPKYRDSALISBURYST', 204),
 ('9THST', 150),
 ('SAGAMOREPKYRD', 1350),
 ('FAIRINGTONAVESOUTHST', 260),
 (None, 126),
 ('BECKLN', 104),
 ('FARABEEDRSOUTHST', 163),
 ('PARKEASTBLVDSOUTHST', 146),
 ('SAGAMOREPKYRDSOUTHST', 337),
 ('EARLAVEMAINST', 201),
 ('I65', 131),
 ('TEALRD', 237),
 ('EASTLANDDRSOUTHST', 218),
 ('CONCORDRDVETERANSMEMORIALPKYRD', 144),
 ('SAGAMOREPKYRDYEAGERRD', 114),
 ('SR38E', 250),
 ('CREASYLNSOUTHST', 553),
 ('SOUTHST', 454),
 ('CUMBERLANDAVESAGAMOREPKYRD', 170),
 ('BRADYLNCONCORDRD', 187),
 ('9THSTSALEMST', 108),
 ('NIGHTHAWKDRSAGAMOREPKYRD', 100),
 ('MAINSTSAGAMOREPKYRD', 107),
 ('MCCARTYLNSAGAMOREPKYRD', 144),
 ('VETERANSMEMORIALPKYRD', 284),
 ('NORTHWESTERNAVESTADIUMAVE', 135),
 ('COMMERCEDR', 149),
 ('RIVERRDWILLIAMSST', 103),
 ('18THSTSOUTHST', 119),
 ('EARLAVESOUTHST', 153),
 ('STATEST', 200),
 ('18THSTTEALRD', 195),
 ('GREENBUSHST', 131),
 ('GRANTST', 173),
 ('STATESTTAPAWINGODR', 160),
 ('SAGAMOREPKWY', 106),
 ('ELMWOODAVE', 102),
 ('GREENBUSHSTSAGAMOREPKYRD', 131),
 ('18THST', 146),
 ('SHENANDOAHDRSOUTHST', 130),
 ('SAGAMOREPKYRDSCHUYLERAVE', 134),
 ('SR38EVETERANSMEMORIALPKYRD', 237),
 ('CREASYLN', 477),
 ('CREASYLNSR38E', 246),
 ('NORTHWESTERNAVE', 325),
 ('CREASYLNGREENBUSHST', 106),
 ('DUNCANRDSAGAMOREPKYRD', 309),
 ('RIVERRDSTATEST', 135),
 ('SAGAMOREPKYRDSR38E', 108),
 ('CHAUNCEYAVE', 117),
 ('OLDSR25RDSR25N', 107),
 ('RIVERRDTAPAWINGODR', 169),
 ('US52SVETERANSMEMORIALPKYRD', 139),
 ('NORTHWESTERNAVEYEAGERRD', 107),
 ('SAGAMOREPKYRDUNIONST', 167),
 ('KOSSUTHSTSAGAMOREPKYRD', 139),
 ('18THSTVETERANSMEMORIALPKYRD', 103),
 ('22NDSTTEALRD', 106),
 ('MAINSTMCCARTYLN', 130),
 ('CREASYLNMCCARTYLN', 231),
 ('I65SOUTHST', 118),
 ('SAGAMOREPKYRDTEALRD', 131),
 ('MAINST', 104)])],
 [('Vigo',
 [('S3RDST', 150),
 ('SUS41HWY', 140),
 ('WABASHAVE', 301),
 ('BROWNAVEWABASHAVE', 108),
 ('25THSTWABASHAVE', 162)])],
 [('Madison',
 [('SCATTERFIELDRD', 187),
 ('MAINST', 174),
 ('DANPATCHCIR', 128),
 ('SSCATTERFIELDRD', 427),
 ('CHARLESSTSCATTERFIELDRD', 160),
 ('38THSTSCATTERFIELDRD', 107),
 ('53RDSTSCATTERFIELDRD', 220),
 

In [52]:
df_dead = df.filter(df['DEADNMB'] > 0)
df_dead.count()

Out[72]: 9784